In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import IPython
import seaborn as sns
import pylab as pl
from tqdm import tqdm

In [7]:
class VacuumCleaner():
    def __init__(self, size=5, sym_empty='░░ ', sym_trash='▓▓ ',
                 sym_vacuum='〄 '):
        
        self.size = size
        self.sym_empty = sym_empty
        self.sym_trash = sym_trash
        self.sym_vacuum = sym_vacuum
        
        
        self.matrix_trash = np.zeros((self.size, self.size))
        self.matrix_vacuum = np.zeros((self.size, self.size))
        
        
        
        self.string_mapping = np.vectorize(lambda s: self.sym_empty if s == 0
                                    else self.sym_trash if s == 1
                                    else self.sym_vacuum)
        self.field = np.zeros((self.size, self.size))
        
        
        self.v_path_indexes = np.array([[1, 1],[1, 2],[1, 3],[2, 3],[3, 3],[3, 2],[3, 1],[2, 1]])
        
        
        self.directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
        
        self.speed = 0.1
        self.style = 'stat'
        
    def output_iteration(self, i):
        if (i%999==0) & (self.style!='print'):
            IPython.display.clear_output(wait=True)
            print('Крок {:>4d} '.format(i + 1))
            self.calculate_trash()
            print('Кількість сміття на полі: {:.1f}'.format(self.trash))
            if self.style=='print':
                self.print_field()
            time.sleep(self.speed)
        if(self.style=='print'):
            IPython.display.clear_output(wait=True)
            print('Крок {:>4d} '.format(i + 1))
            self.calculate_trash()
            print('Кількість сміття на полі: {:.1f} одиниць'.format(self.trash))
            if self.style=='print':
                self.print_field()
            time.sleep(self.speed)
        self.calculate_trash()
        return self.trash
    
    def print_field(self):
        print('\n'.join(''.join(str(cell) for cell in row) for row in self.string_mapping(self.field)))
    
    def calculate_trash(self):
        self.trash = (self.field == 1).sum()
    
    def check_near_trash(self):
        np.random.shuffle(self.directions)
        
        self.stepped = False
        min_distance = self.size * self.size
        
        for direction in self.directions:
            possible_v_pos = tuple(np.array(self.v_pos) + np.array(direction))
            
            if possible_v_pos[0] in range(self.size) and possible_v_pos[1] in range(self.size) and self.field[possible_v_pos] == 1:
                self.prev = self.v_pos #remember previos state
                self.field[self.v_pos] = 0
                self.field[possible_v_pos] = 2
                self.v_pos = possible_v_pos
                self.stepped =  True
                break
            
            if (possible_v_pos==self.prev) or (possible_v_pos[0] not in range(self.size)) or (possible_v_pos[1] not in range(self.size)):
                continue
            
            if self.path_type=='1':   
                if np.linalg.norm(abs(np.array(self.target) - np.array(possible_v_pos))) < min_distance:
                    min_distance = np.linalg.norm(abs(np.array(self.target) - np.array(possible_v_pos)))
                    self.best_v_pos = possible_v_pos
        
                
    def is_vacuum_on_his_path(self):
        return((self.v_path * self.field == 2).any())
    
    def run_process(self, iterations=100, speed=0.1,
                    path_type='right', style='stat'):
        self.iterations = iterations
        self.path_type = path_type
        self.speed = speed
        self.style = style

        self.o = 0
        
        self.v_pos = (0,0)
        self.field[self.v_pos] = 2
        res=0 
        self.prev = (0,0)
        for i in range(self.iterations):
            
            self.target = self.v_path_indexes[self.o]
            if tuple(self.v_pos)==tuple(self.target):
                if self.o==7:
                    self.o=0
                else:
                    self.o+=1
                self.target = self.v_path_indexes[self.o]
            
            
            self.free_field_indexes = np.array(np.where(self.field == 0)).transpose()
            index_to_fill = random.choice(self.free_field_indexes)
            if np.random.rand() > 0.5:
                self.field[index_to_fill[0],index_to_fill[1]] = 1
                self.matrix_trash[index_to_fill[0],index_to_fill[1]]+=1
            
            # output
            res = self.output_iteration(i)
            
            # check trash
            # move and clear if trash is near with little trick
            self.check_near_trash()
            
            # move to/along vacuum path if there isn't any trash
            if not self.stepped:
                self.prev = self.v_pos #remember previos state
                self.field[self.v_pos] = 0
                self.field[self.best_v_pos] = 2
                self.matrix_vacuum[self.best_v_pos]+=1
                self.v_pos = self.best_v_pos 
            
            # output
            res = self.output_iteration(i)
        return res

# Observation tests

In [8]:
v = VacuumCleaner(5)
v.run_process(iterations=100, speed=0.05, path_type='1', style='print')
#pl.clf()

Крок  100 
Кількість сміття на полі: 4.0 одиниць
░░ ░░ ░░ ░░ ░░ 
░░ ░░ ░░ 〄 ░░ 
░░ ░░ ░░ ░░ ░░ 
▓▓ ░░ ░░ ░░ ░░ 
░░ ▓▓ ▓▓ ░░ ▓▓ 


4

In [9]:
import numpy as np; np.random.seed(0)
import seaborn as sns; 
sns.set()

In [ ]:
trash = []
vacuum = []
avg_res = []

# running 1000 times the vacuum algorithm
for i in range(1000):
    v = VacuumCleaner(5)
    avg_res.append(v.run_process(iterations=100, speed=0, path_type='1'))
    trash.append(v.matrix_trash)
    vacuum.append(v.matrix_vacuum)

# for analytics
mean_trash = np.zeros((5, 5))
for i in range(5):
    for j in range(5):
        mean_trash[i,j] = np.mean([hh[i,j] for hh in trash])
        
mean_vacuum = np.zeros((5, 5))
for i in range(5):
    for j in range(5):
        mean_vacuum[i,j] = np.mean([hh[i,j] for hh in vacuum])

print(np.mean(avg_res))
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set()
ax = sns.heatmap(mean_trash)
plt.title('Trash distribution')

Крок    1 
Кількість сміття на полі: 1.0


In [ ]:
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set()
ax = sns.heatmap(mean_vacuum)
plt.title('vacuum distribution')

# Algorithm without memory result
### Average number of trash is approx. 4.8